## Log Data Exporter - Independent Sensor CSVs
This data exporter takes a log file and exports it to a series of CSV files, one per sensor.

Since each sensor is exported separately, these CSV files will not have interpolated data values, only the raw reported sensor values.

In [ ]:
import numpy as np
import pandas
import platypus.io.logs
import os

In [ ]:
# Import the data from the specified logfile
log_filename = '../logs/platypus_20161126_161922.txt'
data = platypus.io.logs.load(log_filename)

# To selectively export certain sensors, change this value to an array of names.
# By default, all sensors are exported (meaning they are also interpolated).
sensor_names = None
# sensor_names = ['ATLAS_DO', 'ES2']

In [ ]:
def export_csv(data, sensor_name, output_filename):
    """
    Export a CSV for a particular sensor in a logfile.

    :param data: a loaded Platypus data log
    :param sensor_name: the name of a sensor in the data log
    :param output_file: the filename to use for the output CSV
    """
    # Extract position of boat and remove duplicate entries.
    data["pose"]["time"] = data["pose"].index
    position = data['pose'][['time', 'latitude', 'longitude']].drop_duplicates(subset='time', keep='first')
    position = position[['latitude', 'longitude']]  # remove unneeded time column after de-duplication

    # Check if the sensor name exists in the data log.
    if sensor_name not in data.keys():
        raise ValueError("'{:s}' sensor was not found in the log file."
                         .format(sensor_name))

    # Put together all the sensor data we are looking for.
    sensor_data = data[sensor_name].copy()

    # Find the position for each sensor reading.
    sensor_position = position.reindex(sensor_data.index, method='nearest')
    output = pandas.concat((sensor_position, sensor_data), axis=1)

    # Fill in missing values with last known values.
    output = output.apply(pandas.Series.interpolate, method='nearest')
    output['epoch_time'] = output.index.astype(np.int64)
    
    # Output the file as a CSV in the same directory as the original logfile.
    output.to_csv(output_filename, index=True)

In [ ]:
# If no sensor is specified, try to export all detected sensors.
if not sensor_names:
    sensor_names = [k for k in data.keys() if k not in ['pose', 'BATTERY']]

# Iterate through and export each sensor as the original logfile name + sensor_name
for sensor_name in sensor_names:
    output_filename = os.path.splitext(log_filename)[0] + '_' + str(sensor_name) + '.csv'
    export_csv(data, sensor_name, output_filename)